##### import

In [17]:
from cvzone.ColorModule import ColorFinder
import cvzone
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle as pickle

##### Kalmann Filter

In [ ]:
class KalmanFilter(object):
    def __init__(self, dt, u_x,u_y, u_z, std_acc, x_std_meas, y_std_meas, z_std_meas):
        """
        :param dt: sampling time (time for 1 cycle)
        :param u_x: acceleration in x-direction
        :param u_y: acceleration in y-direction
        :param std_acc: process noise magnitude
        :param x_std_meas: standard deviation of the measurement in x-direction
        :param y_std_meas: standard deviation of the measurement in y-direction
        """

        # Define sampling time
        self.dt = dt

        # Define the  control input variables
        self.u = np.matrix([u_x,u_y,u_z])

        # Intial State [x, dx, y, dy, z, dz]
        self.x = np.matrix([[0], 
                            [0], 
                            [0], 
                            [0], 
                            [0], 
                            [0]])

        # Define the State Transition Matrix A
        self.F = np.matrix([[1, self.dt, 0, 0, 0, 0],
                            [0, 1, 0, 0, 0, 0],
                            [0, 0, 1, self.dt, 0, 0],
                            [0, 0, 0, 1, 0, 0],
                            [0, 0, 0, 0, 1, self.dt],
                            [0, 0, 0, 0, 0, 1]])

        # Define the Control Input Matrix B
        self.G = np.matrix([[(self.dt**2)/2],
                            [self.dt],
                            [(self.dt**2)/2],
                            [self.dt],
                            [(self.dt**2)/2],
                            [self.dt]])

        # Define Measurement Mapping Matrix
        self.H = np.matrix([[1, 0, 0, 0, 0, 0],
                            [0, 0, 1, 0, 0, 0],
                            [0, 0, 0, 0, 1, 0]])

        #Initial Process Noise Covariance
        self.Q = np.matrix([[(self.dt**4)/4, (self.dt**3)/2, 0, 0, 0, 0],
                            [(self.dt**3)/2, self.dt**2, 0, 0, 0, 0],
                            [0, 0, (self.dt**4)/4, (self.dt**3)/2, 0, 0],
                            [0, 0, (self.dt**3)/2, self.dt**2, 0, 0],
                            [0, 0, 0, 0, (self.dt**4)/4, (self.dt**3)/2],
                            [0, 0, 0, 0, (self.dt**3)/2, self.dt**2],]) * std_acc**2

        #Initial Measurement Noise Covariance
        self.R = np.matrix([[x_std_meas**2, 0, 0],
                           [0, y_std_meas**2, 0],
                           [0, 0, z_std_meas**2]])

        #Initial Covariance Matrix
        self.P = np.eye(self.F.shape[1])

    def predict(self):
        # Refer to :Eq.(9) and Eq.(10)  in https://machinelearningspace.com/object-tracking-simple-implementation-of-kalman-filter-in-python/?preview_id=1364&preview_nonce=52f6f1262e&preview=true&_thumbnail_id=1795

        # Update time state
        #x_k =Ax_(k-1) + Bu_(k-1)     Eq.(9)
        # print(np.shape(np.dot(self.F, self.x)), np.dot(self.F, self.x))
        # print(np.shape(np.dot(self.G, self.u)), np.dot(self.G, self.u))
        # self.x = np.dot(self.F, self.x) + np.dot(self.G, self.u)
        # print(np.shape(self.x), self.x)
        self.x = np.dot(self.F, self.x) # disable control input

        # Calculate error covariance
        # P= A*P*A' + Q               Eq.(10)
        self.P = np.dot(np.dot(self.F, self.P), self.F.T) + self.Q
        return (self.x[0], self.x[2], self.x[4])

    def update(self, z):

        # Refer to :Eq.(11), Eq.(12) and Eq.(13)  in https://machinelearningspace.com/object-tracking-simple-implementation-of-kalman-filter-in-python/?preview_id=1364&preview_nonce=52f6f1262e&preview=true&_thumbnail_id=1795
        # S = H*P*H'+R
        S = np.dot(self.H, np.dot(self.P, self.H.T)) + self.R

        # Calculate the Kalman Gain
        # K = P * H'* inv(H*P*H'+R)
        K = np.dot(np.dot(self.P, self.H.T), np.linalg.inv(S))  #Eq.(11)

        self.x = np.round(self.x + np.dot(K, (z - np.dot(self.H, self.x))))   #Eq.(12)

        I = np.eye(self.H.shape[1])
        # temp = I - (K * self.H)
        temp = I - (K * self.H)

        # Update error covariance matrix
        self.P = temp * self.P * np.linalg.inv(temp) + K*self.R*K.T  #Eq.(13)
        return (self.x[0], self.x[2], self.x[4])


In [ ]:
class KalmanFilter1D(object):
    def __init__(self, dt, u, std_acc, std_meas):
        self.dt = dt
        self.u = u
        self.std_acc = std_acc

        self.F = np.matrix([[1, self.dt],
                            [0, 1]])
        self.G = np.matrix([[(self.dt**2)/2], 
                            [self.dt]])

        self.H = np.matrix([[1, 0]])

        self.Q = np.matrix([[(self.dt**4)/4, (self.dt**3)/2],
                            [(self.dt**3)/2, self.dt**2]]) * self.std_acc**2

        self.R = std_meas**2

        self.P = np.eye(self.F.shape[1])
        
        self.x = np.matrix([[0], 
                            [0]])

        # print(self.Q)


    def predict(self):
        # Ref :Eq.(9) and Eq.(10)

        # Update time state
        self.x = np.dot(self.F, self.x) + np.dot(self.G, self.u)

        # Calculate error covariance
        # P= A*P*A' + Q
        self.P = np.dot(np.dot(self.F, self.P), self.F.T) + self.Q
        return self.x[0]

    def update(self, z):
        # Ref :Eq.(11) , Eq.(11) and Eq.(13)
        # S = H*P*H'+R
        S = np.dot(self.H, np.dot(self.P, self.H.T)) + self.R

        # Calculate the Kalman Gain
        # K = P * H'* inv(H*P*H'+R)
        K = np.dot(np.dot(self.P, self.H.T), np.linalg.inv(S))  # Eq.(11)

        self.x = np.round(self.x + np.dot(K, (z - np.dot(self.H, self.x))))  # Eq.(12)

        I = np.eye(self.H.shape[1])
        self.P = (I - (K * self.H)) * self.P  # Eq.(13)
        # print("self.x : {0}".format(self.x))
        return self.x[0]

##### object detection + tracking

In [10]:
def objectTrackingKalmanFilter(videoPath, outputFileName, isLive, isShow):
    if(not(isLive)):
        # Create opencv video capture object
        # cap = cv2.VideoCapture('video/tes_jarak_bola.mp4')
        cap = cv2.VideoCapture(videoPath)
    else:
        cap = cv2.VideoCapture(0)

    cap.set(3, 1280)
    cap.set(4, 720)
    
    myColorFinder = ColorFinder(0)
    # warnaBolaHsv = {
    #     "hijau muda"        : {'hmin': 81, 'smin': 109, 'vmin': 92, 'hmax': 179, 'smax': 244, 'vmax': 255},    # bola hijau muda
    #     "kuning kehijauan"  : {'hmin': 23, 'smin': 41, 'vmin': 126, 'hmax': 62, 'smax': 174, 'vmax': 227},    # bola kuning kehijauan
    #     "hijau tua"         : {'hmin': 35, 'smin': 42, 'vmin': 123, 'hmax': 73, 'smax': 190, 'vmax': 255},    # bola hijau tua
    #     # "biru muda"         : {'hmin': 94, 'smin': 47, 'vmin': 118, 'hmax': 109, 'smax': 204, 'vmax': 255},   # bola biru muda
    #     # "hijau telur asin"  : {'hmin': 84, 'smin': 61, 'vmin': 137, 'hmax': 105, 'smax': 171, 'vmax': 255},   # bila hijau telur asin
    #     # "biru telur unta"   : {'hmin': 83, 'smin': 69, 'vmin': 90, 'hmax': 108, 'smax': 255, 'vmax': 255},    # bola biru telur unta
    #     "oranye"            : {'hmin': 13, 'smin': 119, 'vmin': 165, 'hmax': 23, 'smax': 241, 'vmax': 255},     # bola oranye
    #     # "pink"              : {'hmin': 149, 'smin': 20, 'vmin': 164, 'hmax': 179, 'smax': 79, 'vmax': 255},   # bola pink
    #     "ungu"              : {'hmin': 140, 'smin': 41, 'vmin': 107, 'hmax': 162, 'smax': 148, 'vmax': 255},   # ungu
    #     # "krem"              : {'hmin': 0, 'smin': 0, 'vmin': 155, 'hmax': 179, 'smax': 26, 'vmax': 255},      # bola krem -> tapi karena cahaya malah mirip putih
    # }

    # tes di wisnu
    warnaBolaHsv = {
        "hijau muda"        : {'hmin': 49, 'smin': 93, 'vmin': 126, 'hmax': 80, 'smax': 253, 'vmax': 255},    # bola hijau muda
        # "kuning kehijauan"  : {'hmin': 23, 'smin': 41, 'vmin': 126, 'hmax': 62, 'smax': 174, 'vmax': 227},    # bola kuning kehijauan
        # "hijau tua"         : {'hmin': 35, 'smin': 42, 'vmin': 123, 'hmax': 73, 'smax': 190, 'vmax': 255},    # bola hijau tua
        # "biru muda"         : {'hmin': 94, 'smin': 47, 'vmin': 118, 'hmax': 109, 'smax': 204, 'vmax': 255},   # bola biru muda
        # "hijau telur asin"  : {'hmin': 84, 'smin': 61, 'vmin': 137, 'hmax': 105, 'smax': 171, 'vmax': 255},   # bila hijau telur asin
        # "biru telur unta"   : {'hmin': 83, 'smin': 69, 'vmin': 90, 'hmax': 108, 'smax': 255, 'vmax': 255},    # bola biru telur unta
        "oranye"            : {'hmin': 12, 'smin': 129, 'vmin': 217, 'hmax': 39, 'smax': 255, 'vmax': 255},     # bola oranye
        # "pink"              : {'hmin': 149, 'smin': 20, 'vmin': 164, 'hmax': 179, 'smax': 79, 'vmax': 255},   # bola pink
        "ungu"              : {'hmin': 142, 'smin': 101, 'vmin': 138, 'hmax': 171, 'smax': 243, 'vmax': 255},   # ungu
        # "krem"              : {'hmin': 0, 'smin': 0, 'vmin': 155, 'hmax': 179, 'smax': 26, 'vmax': 255},      # bola krem -> tapi karena cahaya malah mirip putih
    }

    kalmanFilterBola = {}
    kalmanFilterSupportingAttributes = {}
    kalmanFilterHasil = {}
    for warnaBola in warnaBolaHsv:
        kalmanFilterBola[warnaBola] = {"KF_X" : KalmanFilter1D(0.1, 1, 1, 0.1), "KF_Y" : KalmanFilter1D(0.1, 1, 1, 0.1), "KF_Z" : KalmanFilter1D(0.1, 1, 1, 0.1)}
        kalmanFilterSupportingAttributes[warnaBola] = {"is_active" : False, "is_new" : True, "frame_since_last_detected" : 0}
        kalmanFilterHasil[warnaBola] = []
    MAXFRAMESINCEDETECTED = 10

    # hsvVals = {'hmin': 33, 'smin': 72, 'vmin': 126, 'hmax': 58, 'smax': 255, 'vmax': 255} # hijau
    # hsvVals = {'hmin': 23, 'smin': 41, 'vmin': 126, 'hmax': 62, 'smax': 174, 'vmax': 227} # bola kuning kehijauan
    hsvVals = {'hmin': 35, 'smin': 42, 'vmin': 123, 'hmax': 73, 'smax': 124, 'vmax': 255} # bola hijau tua
    # hsvVals = {'hmin': 69, 'smin': 62, 'vmin': 50, 'hmax': 107, 'smax': 255, 'vmax': 117} # hijau tua
    # hsvVals = {'hmin': 6, 'smin': 42, 'vmin': 53, 'hmax': 32, 'smax': 206, 'vmax': 156} # kulit

    #define kernel size  
    kernel = np.ones((7,7),np.uint8)

    # #Create KalmanFilter object KF
    # KF = KalmanFilter(0.1, 1, 1, 1, 1, 0.1, 0.1, 0.1)
    # KF_X = KalmanFilter1D(0.1, 1, 1, 0.1)
    # KF_Y = KalmanFilter1D(0.1, 1, 1, 0.1)
    # KF_Z = KalmanFilter1D(0.1, 1, 1, 0.1)

    # init object
    x, y, z = 0, 0, 0


    # open file
    # fileResult1 = open(outputFileName, "w")
    # for warnaBola in warnaBolaHsv:
    #     fileResult1.write(warnaBola + ", ")
    # fileResult1.write("\n")

    try:
        while(True):
            success, img = cap.read()
            allMask = []
            # h, w, _ = img.shape
            for warnaBola in warnaBolaHsv:
                hsvVals = warnaBolaHsv[warnaBola]
                KF_1D = kalmanFilterBola[warnaBola]
                kalmanFilterSupportingAttributesLocal = kalmanFilterSupportingAttributes[warnaBola]
                kalmanFilterHasilLokal = kalmanFilterHasil[warnaBola]
                imageColor, mask = myColorFinder.update(img, hsvVals)
                # print("mask shape:", np.shape(mask), "value:", np.unique(mask))

                # Remove unnecessary noise from mask
                mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
                mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

                contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                if(len(contours) > 0):
                    if(not(kalmanFilterSupportingAttributesLocal["is_active"])):
                        kalmanFilterBola[warnaBola] = {"KF_X" : KalmanFilter1D(0.1, 1, 1, 0.1), "KF_Y" : KalmanFilter1D(0.1, 1, 1, 0.1), "KF_Z" : KalmanFilter1D(0.1, 1, 1, 0.1)}
                        KF_1D = kalmanFilterBola[warnaBola]
                        kalmanFilterSupportingAttributesLocal["is_active"] = True

                    kalmanFilterSupportingAttributesLocal["frame_since_last_detected"] = 0

                    biggestContour = max(contours, key = cv2.contourArea)
                    cv2.drawContours(img, biggestContour, -1, (0,255,0), 3)

                    # bisa dipake buat approximate lokasi di video depth
                    # https://stackoverflow.com/questions/69637673/finding-points-within-a-contour-using-opencv
                    # https://stackoverflow.com/questions/70438811/reading-frames-from-two-video-sources-is-not-in-sync-opencv
                    x,y,w,h = cv2.boundingRect(biggestContour) 
                    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

                    # Predict
                    # (x, y, z) = KF.predict()
                    x, y, z = KF_1D["KF_X"].predict(), KF_1D["KF_Y"].predict(), KF_1D["KF_Z"].predict()

                    # Update
                    M = cv2.moments(biggestContour)
                    if(M['m00'] != 0):
                        cx = int(M['m10']/M['m00'])
                        cy = int(M['m01']/M['m00'])
                    else:
                        cx = x
                        cy = y
                    # (x1, y1, z1) = KF.update([cx, cy, 0])
                    x1, y1, z1 = KF_1D["KF_X"].update(cx), KF_1D["KF_Y"].update(cy), KF_1D["KF_Z"].update(0)

                    # Draw a rectangle as the predicted object position
                    cv2.rectangle(img, (int(x - 15), int(y - 15)), (int(x + 15), int(y + 15)), (0, 0, 255), 2)
                    cv2.rectangle(img, (int(x - 15), int(y - 15)), (int(x + 15), int(y + 15)), (0, 255, 255), 2)
                else:
                    if(kalmanFilterSupportingAttributesLocal["frame_since_last_detected"] > MAXFRAMESINCEDETECTED):
                        kalmanFilterSupportingAttributesLocal["is_active"] = False
                    else:
                        kalmanFilterSupportingAttributesLocal["frame_since_last_detected"] += 1
                        # Predict
                        # (x, y, z) = KF.predict()
                        x, y, z = KF_1D["KF_X"].predict(), KF_1D["KF_Y"].predict(), KF_1D["KF_Z"].predict()

                        cx = x
                        cy = y
                        x1, y1, z1 = KF_1D["KF_X"].update(cx), KF_1D["KF_Y"].update(cy), KF_1D["KF_Z"].update(0)

                        # Draw a rectangle as the predicted object position
                        cv2.rectangle(img, (int(x - 15), int(y - 15)), (int(x + 15), int(y + 15)), (0, 0, 255), 2)
                        cv2.rectangle(img, (int(x - 15), int(y - 15)), (int(x + 15), int(y + 15)), (0, 255, 255), 2)
                if(kalmanFilterSupportingAttributesLocal["is_active"]):
                    cv2.putText(img, warnaBola, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 0, 0), 1, cv2.LINE_AA)
                    cv2.putText(img, str(cv2.contourArea(biggestContour)), (int(x), int(y)-25), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 0, 0), 1, cv2.LINE_AA)
                    kalmanFilterHasilLokal.append([x[0,0], y[0,0]])
                    # fileResult1.write("{:.3f}, {:.3f}, ".format(x[0,0], y[0,0]))
                    # print("nilai: {:.3f}, {:.3f}, ".format(x[0,0], y[0,0]))
                    # print("shape: {}, {}, ".format(np.shape(x[0,0]), np.shape(y[0,0])))
                else:
                    # fileResult1.write("-, -, ")
                    kalmanFilterHasilLokal.append([-1, -1])
                allMask.append(mask)
            # fileResult1.write("\n")

            if(isShow):
                # imageStack = cvzone.stackImages([img, mask], 2, 0.5)
                cv2.imshow("imageOri", img)

                imageStack = cvzone.stackImages(allMask, 2, 0.5)
                cv2.imshow("imageStack", imageStack)


            if cv2.waitKey(2) & 0xFF == ord('q'):
                    cap.release()
                    cv2.destroyAllWindows()
                    break

            cv2.waitKey(1)
    except Exception as e:
        print("error: ", e)
        cap.release()
        cv2.destroyAllWindows()
    return kalmanFilterHasil
    # fileResult1.close()


In [11]:
hasilObjectTracking = {}
hasilObjectTracking["0"] = objectTrackingKalmanFilter('video/tes_di_wisnu_3.mp4', "output1.txt", False, False)
hasilObjectTracking["1"] = objectTrackingKalmanFilter('video/tes_di_wisnu_3.mp4', "output2.txt", False, False)
hasilObjectTracking["2"] = objectTrackingKalmanFilter('video/tes_di_wisnu_3.mp4', "output3.txt", False, False)
print(hasilObjectTracking)

error:  OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

error:  OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

error:  OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

{'0': {'hijau muda': [[0.005000000000000001, 0.005000000000000001], [201.005, 386.805], [203.005, 390.905], [203.305, 390.605], [202.70499999999998, 389.705], [202.405, 389.205], [202.20499999999998, 388.905], [202.105, 388.705], [202.005, 388.505], [201.905, 387.305], [201.905, 386.105], [201.805, 386.005], [201.70499999999998, 385.905], [200.70499999999998, 385.805], [200.70499999999998, 385.705], [200.70499999999998, 384.705], [200.70499999999998, 384.705], [200.70499999999998, 384.

In [19]:
fileResultPickle1 = open("outputPickle", "ab")
pickle.dump(hasilObjectTracking, fileResultPickle1)
fileResultPickle1.close()

fileResultPickle2 = open("outputPickle", "rb")
loadPickle = pickle.load(fileResultPickle2)
fileResultPickle2.close()

In [21]:
print(hasilObjectTracking.keys())
print(hasilObjectTracking["0"].keys())
print(hasilObjectTracking["1"].keys())
print(hasilObjectTracking["2"].keys())

print(loadPickle.keys())
print(loadPickle["0"].keys())
print(loadPickle["1"].keys())
print(loadPickle["2"].keys())
print(loadPickle)

dict_keys(['0', '1', '2'])
dict_keys(['hijau muda', 'oranye', 'ungu'])
dict_keys(['hijau muda', 'oranye', 'ungu'])
dict_keys(['hijau muda', 'oranye', 'ungu'])
dict_keys(['0', '1', '2'])
dict_keys(['hijau muda', 'oranye', 'ungu'])
dict_keys(['hijau muda', 'oranye', 'ungu'])
dict_keys(['hijau muda', 'oranye', 'ungu'])
{'0': {'hijau muda': [[0.005000000000000001, 0.005000000000000001], [201.005, 386.805], [203.005, 390.905], [203.305, 390.605], [202.70499999999998, 389.705], [202.405, 389.205], [202.20499999999998, 388.905], [202.105, 388.705], [202.005, 388.505], [201.905, 387.305], [201.905, 386.105], [201.805, 386.005], [201.70499999999998, 385.905], [200.70499999999998, 385.805], [200.70499999999998, 385.705], [200.70499999999998, 384.705], [200.70499999999998, 384.705], [200.70499999999998, 384.705], [200.70499999999998, 384.705], [200.805, 384.705], [200.905, 384.705], [202.105, 384.705], [203.305, 384.705], [204.405, 384.705], [205.505, 384.705], [206.605, 384.705], [207.805, 384.7